In [ ]:
# Import necessary packages
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [ ]:
# Initialize instance of H2O
h2o.init()

In [ ]:
# If possible download from the s3 link and change the path to the dataset
path = "http://h2o-public-test-data.s3.amazonaws.com/bigdata/laptop/lending-club/LoanStats3a.csv"

In [ ]:
# Specify some column types to "String" that we want to munge later
types = {"int_rate":"string", "revol_util":"string", "emp_length":"string", 
         "earliest_cr_line":"string", "issue_d":"string", "last_credit_pull_d":"factor", "verification_status":"string"}

In [ ]:
# Task 1: Import the file and look at the frame
data = h2o.import_file(path=path, col_types= types)
data.describe()

In [ ]:
# Task 2: Look at the levels int he response column, "loan_status"
# Hint: Use .table() function on the response column
data["loan_status"].table()

In [ ]:
data.show()

In [ ]:
# Task 3: Bin the response variable to good/bad oans only, use your best judgement for what qualifies as a good/bad loan
# Create a new column called "bad_loan" which should be a binary variable
# Hint: You can turn the bad_loan columm into a factor using .asfactor()
data["bad_loan"] = data["loan_status"].isin(["Charged Off", "Default", 
                                                  "Does not meet the credit policy.  Status:Charged Off"])
data ["bad_loan"] = data["bad_loan"].asfactor()

In [ ]:
data["int_rate"].show()

In [ ]:
# Task 4: String munging to clean string columns before converting to numeric
# Hint: Column that need munging include "int_rate," "revol_util," "emp_length"

#### Example for int_rate using gsub, trim, asnumeric ####
data ["int_rate"] = data["int_rate"].gsub(pattern = "%", replacement = "") # strip %
data [ "int_rate"] = data["int_rate"].trim() # trim ws
data ["int_rate"] = data["int_rate"].asnumeric() #change to a numeric 
data["int_rate"].show()

In [ ]:
data["revol_util"].show()

In [ ]:
# Now try for revol_util yourself
data["revol_util"] = data["revol_util"].gsub(pattern = "%", replacement = "")
data["revol_util"] = data["revol_util"].trim() 
data["revol_util"] = data["revol_util"].asnumeric() 
data["revol_util"].show()

In [ ]:
data["emp_length"].show()

In [ ]:
# Now we're going to clean up emp_length
# Use gsub to remove " year" and " years" also translate n/a to "" 
data ["emp_length"] = data["emp_length"].gsub(pattern = "([ ]*+[a-zA-Z].*)|(n/a)", 
                                              replacement = "") 

# Use trim to remove any trailing spaces 
data ["emp_length"] = data["emp_length"].trim()

In [ ]:
data ["emp_length"].show()

In [ ]:
# Use sub to convert < 1 to 0.5 years and do the same for 10 + to 10
# Hint: Be mindful of spaces between characters
data ["emp_length"] = data["emp_length"].gsub(pattern = "< 1", replacement = "0.5")
data ["emp_length"] = data["emp_length"].gsub(pattern = "10\\+", replacement = "10")
data ["emp_length"] = data["emp_length"].asnumeric()
data ["emp_length"].show()

In [ ]:
data["earliest_cr_line"].show()

In [ ]:
# Task 5: Extract month and year from earliest_cr_line and make two new columns called 
# earliest_cr_month and earliest_cr_year
data["earliest_cr_month"] = data["earliest_cr_line"].strsplit(pattern = "-")[0]
data["earliest_cr_year"] = data["earliest_cr_line"].strsplit(pattern = "-")[1]
data["earliest_cr_year"] = data["earliest_cr_line"].asnumeric()
data["earliest_cr_year"] = data["earliest_cr_line"].strsplit(pattern = "-")[1].asnumeric()
data["earliest_cr_year"].show()

In [ ]:
data["issue_d"].strsplit(pattern = "-")[0].show()

In [ ]:
# Task 6: Extract month and year from issue_d and make two new columns called issue_d_month and issue_d_year
data["issue_d_month"] = data["issue_d"].strsplit(pattern = "-")[0]
data["issue_d_year"] = data["issue_d"].strsplit(pattern = "-")[1]
data["issue_d_year"] = data["issue_d_year"].asnumeric()
data["issue_d_year"].show()

In [ ]:
# Task 7: Create new column called credit_length
# Hint: Do this by subtracting the earliest_cr_year from the issue_d_year
data["credit_length"] = data["issue_d_year"] - data["earliest_cr_year"]
data["credit_length"].show()

In [ ]:
data["verification_status"].show()

In [ ]:
# Task 8: Use the sub function to create two levels from the verification_status column. Ie "verified" and "not verified"
data ["verification_status"] = data["verification_status"].sub(pattern = "VERIFIED - income source", 
                                                               replacement = "verified")
data ["verification_status"] = data["verification_status"].sub(pattern = "VERIFIED - income", 
                                                               replacement = "verified")
data ["verification_status"] = data["verification_status"].asfactor()

In [ ]:
# Task 9: Do a test-train split (80-20)
s = data["int_rate"].runif()
train = data[s <= 0.80]
test  = data[s > 0.80]

In [ ]:
# Task 10: Define your response and predictor variables 
y="bad_loan"
x=["loan_amnt", "credit_length", "revol_util", 
     "home_ownership", "annual_inc", "purpose", "addr_state", "dti",
     "delinq_2yrs", "total_acc", "verification_status", "term"]

In [ ]:
# Task 11: Set parameters for GBM model 
from h2o.estimators.gbm import H2OGradientBoostingEstimator
model = H2OGradientBoostingEstimator(model_id="BadLoanModelPython",
                                       score_each_iteration=True,
                                       ntrees=10,
                                       learn_rate=0.05)

In [ ]:
# Task 12: Build your model and view results 
model.train(x=x, y=y, training_frame=train, validation_frame=test)
model